In [250]:
from __future__ import absolute_import #导入3.x的特征函数
from __future__ import print_function
from imp import reload
import pandas as pd #导入Pandas
import numpy as np #导入Numpy
import jieba #导入结巴分词
import gensim
import h5py
import sys
import re
reload(sys)
#sys.setdefaultencoding('utf-8')

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
#from keras.utils import plot_model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional #包装器，把一个层应用到输入的每一个时间步上
from keras.callbacks import TensorBoard  #TensorBoard是TensorFlow提供的可视化工具，
                                          #该回调函数将日志信息写入TensorBorad，
                                          #使得你可以动态的观察训练和测试指标的图像以及不同层的激活值直方图。
from keras.metrics import binary_accuracy #对二分类问题,计算在所有预测值上的平均正确率
from keras import backend as K            #keras后端
from pickle import dump, load

In [416]:
jieba.enable_parallel(32)
cw = lambda x: list(jieba.cut(str(x))) #定义分词函数

In [417]:
re_sub = lambda x:re.sub("\W", "", x)
re_sub_vec = np.vectorize(re_sub)
#re_sub(text.content[0])

In [418]:
text = list()
for i in range(1,141):
    with open('/home/yuangxue/abstract/news/training%d.txt' % (i)) as f:
        a = f.readlines()
        text.append(''.join(a))
#     with open('/home/yuangxue/abstract/news/training%d.txt' % (i)) as f:
#         a = f.readlines()
#         text.append(''.join(a))
text = pd.DataFrame({'content':text})
text['content'] = re_sub_vec(text['content'])
text['words'] = text['content'].apply(cw) #评论分词

abstract = list()
for i in range(1,141):
    with open('/home/yuangxue/abstract/abstract/training%db.txt' % (i)) as f:
        a = f.readlines()
        abstract.append(''.join(a))
#     with open('/home/yuangxue/abstract/abstract/training%da.txt' % (i)) as f:
#         a = f.readlines()
#         abstract.append(''.join(a))
abstract = pd.DataFrame({'content':abstract})
abstract['content'] = re_sub_vec(abstract['content'])
abstract['words'] = abstract['content'].apply(cw) #评论分词

text_len = 1200
abstract_len = 20 

In [419]:
# wordvec_model = gensim.models.word2vec.Word2Vec.load('/home/yuangxue/word2vec_from_weixin/word2vec/word2vec_wx')
wordvec_model = gensim.models.word2vec.Word2Vec.load('/home/jeffmxh/word2vec/wiki.zh.text.model')
wordvec_weight = wordvec_model.wv.syn0
vocab = dict([(k, v.index) for k, v in wordvec_model.wv.vocab.items()])
vocab_inv = {vocab[w]:w for w in vocab.keys()}
word_embedding_layer = Embedding(
        input_dim=wordvec_weight.shape[0],
        output_dim=wordvec_weight.shape[1],
        weights=[wordvec_weight],
        trainable=False)
word_to_id = lambda word: not (vocab.get(word) is None) and vocab.get(word) or 0 #词->index
words_to_ids = lambda words: list(map(word_to_id, words))       

# text['sent'] = list(sequence.pad_sequences(text['words'], maxlen=text_len, padding='post', truncating='post'))

In [420]:
len(wordvec_model['开心'])

300

In [422]:
text['sent'] = text['words'].apply(words_to_ids)                  #变成432100
text['sent'] = list(sequence.pad_sequences(text['sent'], maxlen=text_len, padding='post', truncating='post'))
abstract['sent'] = abstract['words'].apply(words_to_ids)
abstract['sent'] = list(sequence.pad_sequences(abstract['sent'], maxlen=abstract_len, padding='post', truncating='post'))

In [423]:
def vectorize(sent_list, length, dim = 256):
    vec = np.zeros((length, dim), dtype = float)
    for i, ch in enumerate(sent_list):
        vec[i] = wordvec_model[vocab_inv[ch]]
    return vec

In [424]:
xa = np.zeros((len(text), 1200, 300))
ya = np.zeros((len(abstract), 20, 300))
for i in range(len(text.sent)):
    xa[i] = vectorize(text.sent[i], 1200, 300)
for i in range(len(abstract.sent)): 
    ya[i] = vectorize(abstract.sent[i], 20, 300)

In [425]:
x = xa[::2] #训练集
xt = xa[1::2] #测试集
y = ya[::2] #训练集
yt = ya[1::2] #测试集

In [429]:
model = Sequential()
model.add(LSTM(128, input_shape=(None,300), return_sequences=False))
# model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(RepeatVector(20))
model.add(LSTM(128, return_sequences=True))
# model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(300, activation="linear")))
model.compile(loss="mse", optimizer='adam')

In [430]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_80 (LSTM)               (None, 128)               219648    
_________________________________________________________________
dense_76 (Dense)             (None, 128)               16512     
_________________________________________________________________
repeat_vector_39 (RepeatVect (None, 20, 128)           0         
_________________________________________________________________
lstm_81 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
time_distributed_24 (TimeDis (None, 20, 300)           38700     
Total params: 406,444.0
Trainable params: 406,444.0
Non-trainable params: 0.0
_________________________________________________________________
None


In [431]:
model.fit(xa, ya, validation_data=(xt, yt), epochs=10, batch_size=16)

Train on 140 samples, validate on 70 samples
Epoch 1/10
140/140 [==============================] - 25s - loss: 0.7953 - val_loss: 0.7203
Epoch 2/10
140/140 [==============================] - 20s - loss: 0.7089 - val_loss: 0.6739
Epoch 3/10
140/140 [==============================] - 20s - loss: 0.6860 - val_loss: 0.6689
Epoch 4/10
140/140 [==============================] - 20s - loss: 0.6828 - val_loss: 0.6666
Epoch 5/10
140/140 [==============================] - 20s - loss: 0.6808 - val_loss: 0.6654
Epoch 6/10
140/140 [==============================] - 20s - loss: 0.6796 - val_loss: 0.6648
Epoch 7/10
140/140 [==============================] - 17s - loss: 0.6791 - val_loss: 0.6645
Epoch 8/10
140/140 [==============================] - 17s - loss: 0.6785 - val_loss: 0.6639
Epoch 9/10
140/140 [==============================] - 16s - loss: 0.6782 - val_loss: 0.6636
Epoch 10/10
140/140 [==============================] - 16s - loss: 0.6779 - val_loss: 0.6632


In [432]:
xtest = np.zeros((1, 1200, 300))

xtest[0] = x[0]

b = model.predict(xtest)[0]
# print(y[1])
# print(b)

In [441]:
text.content[0]

'吾城吾乡原乡异域别人的城市在当下的现实语境中故乡是一个语义丰富而模糊的词语比如一年一度的春运大潮显然不仅仅是因为返乡的动因使然在全球化与城市化的进程中我们每一个人都不能不面对从故乡到异乡的流转与变迁即使从未离开过的家园也在高速发展的物质社会中不断蜕变吾城吾乡既是传统承袭的故园情怀也是重新来过的自我定位背后既有个人的生命体验亦是家国命运的折射我们选择了五座城乡八位摄影师以两两对照并持续数年拍摄的影像阐释这一命题摄影就像一只手悄悄地在现实和精神世界中放下地标在喧嚣与骚动中我们每一个人或许都是某种意义上的异乡人而摄影能否成为一个微弱而清晰的声音说看吾城吾乡专题按语李楠深圳南山区的南山村南园村北头村三村连成一片形成深圳最大的城中村这是透过飞机舷窗拍摄到的三村全貌2008年摄张新民图吾城吾乡原乡异域农村包围城市的大迁徙不仅仅是从原乡到异城的地理转移更是物质身份与精神身份的双重置换国家城市化进程中伴随的个人城市化注定是一场身心交战城市可以栖居却未必可以归属乡土无法归去却又藕断丝连所谓的寻根之旅往往是获得了表面零碎的记忆片断之后更为深刻的精神失落被父亲牵引着从城市重返原乡的孙京涛只能以外来者的目光重新打量他最熟悉的人们而远离故里的张新民跟随乡亲们一起见证了中国民工潮最终停靠在一座平空而起的城市深圳因为这样的经历他们平静的镜头里汹涌着跌宕起伏的宿命和无法停止的寻找原乡与异城或许将会是我们越来越难以抉择的矛盾在沙井镇川籍农民工聚居的棚户区春节期间很多人回不了故乡小杂货铺的电视机前经常挤满了老乡1997年摄张新民图别人的城市腊月二十三祭灶王爷老胡两口子百味杂陈汽车票买到了深圳到潼南680元一个人比平常贵一倍但明天终于可以回去过年了来深圳打工10年两口子只回去过两次在南方狭窄的出租屋里过年为的就是攒下钱来帮补儿女正儿八经在城里安家这个目标在2013年就要实现了儿子在重庆买了房春节结婚这样的喜事做父母的当然要提前几天回去但厂里硬是不批假放出话来要走可以自动离职这意味着老胡一个月的工钱是彻底拿不到了深圳的最低工资线是1500元老胡拿1800像他这样的川籍重庆籍农民工在深圳有很多他们靠出卖劳动力吃饭一干就是十几二十年从来不敢想象能住进这里的天价房他们住棚屋住城中村年复一年至今未变1997年我曾在沙井镇拍摄过我的老乡四川农民工聚居的棚户区后来棚户区没有了外来务工人员的落脚地是城中村或者城乡接合

In [439]:
for i in range(20):
    print(wordvec_model.similar_by_vector(x[0][i])[7])

('滨田省', 0.5977662205696106)
('同年', 0.5889389514923096)
('国姓', 0.5328131914138794)
('末世', 0.5393956899642944)
('同年', 0.5889389514923096)
('同年', 0.5889389514923096)
('小城镇', 0.4807368516921997)
('同年', 0.5889389514923096)
('同年', 0.5889389514923096)
('同年', 0.5889389514923096)
('主观', 0.5208644866943359)
('约定俗成', 0.7271884679794312)
('中曾', 0.5745320916175842)
('思乡', 0.4522740840911865)
('同年', 0.5889389514923096)
('同年', 0.5889389514923096)
('抽象概念', 0.7426023483276367)
('同年', 0.5889389514923096)
('同年', 0.5889389514923096)
('主观', 0.5855385661125183)


In [356]:
wordvec_model.similar_by_vector(wordvec_model['！'])

[('！', 1.0),
 ('吧', 0.7221462726593018),
 ('哦', 0.6962027549743652),
 ('啦', 0.6935625076293945),
 ('哟', 0.6884435415267944),
 ('噢', 0.6847633123397827),
 ('~', 0.6815340518951416),
 ('啊', 0.6630978584289551),
 ('!', 0.6576281189918518),
 ('咯', 0.652571439743042)]

In [216]:
x1 = np.array(list(text['sent']))[::2] #训练集
xt1 = np.array(list(text['sent']))[1::2] #测试集
xa1 = np.array(list(text['sent'])) #全集

y1 = np.array(list(abstract['sent']))[::2] #训练集
yt1 = np.array(list(abstract['sent']))[1::2] #测试集
ya1 = np.array(list(abstract['sent'])) #全集

In [217]:
model = Sequential()
model.add(word_embedding_layer)
model.add(LSTM(128, return_sequences=False))
model.add(Dense(128, activation="relu"))
model.add(RepeatVector(100))
model.add(LSTM(128, return_sequences=True))
model.add(TimeDistributed(Dense(256, activation="linear")))
model.compile(loss="mse", optimizer='adam')